# Model (ResNet) - Binary

Within this model, we'll use the same VGG16 model, but reduce the options down to 'Subuaru' and 'Not Subaru'

### Library Imports

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import pickle
import time
import os
import json
import seaborn as sns
import tensorflow as tf
import random
import cv2
import copy

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import torchvision.transforms as transforms



from tensorflow.keras.applications import VGG16,InceptionV3
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.models import Sequential

from keras.applications import VGG16
from keras.applications.resnet import ResNet152
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from datetime import datetime

from pandas import json_normalize
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay,accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, f1_score,classification_report
from sklearn.preprocessing import normalize,StandardScaler,RobustScaler,MinMaxScaler
from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.compose import ColumnTransformer



from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Flatten, Conv2D, MaxPooling2D,LeakyReLU
from tensorflow.keras.regularizers import l1, l2, l1_l2
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import EarlyStopping
from tensorflow.keras.datasets import mnist, cifar10

from tensorflow import keras
from keras.utils import to_categorical
from keras import regularizers
from keras.models import load_model
from keras.layers import GlobalAveragePooling2D
from keras.losses import binary_crossentropy

from scipy.stats import skew
from tqdm import tqdm
from PIL import Image
from sklearn import metrics


### Data Augmentation

In [2]:
# #Remove comment to run again
# datagen = ImageDataGenerator(
#     rotation_range=20,
# #     width_shift_range=0.15,
# #     height_shift_range=0.15,
#     shear_range=0.1,
#     zoom_range=0.1,
#     horizontal_flip=True,
#     vertical_flip=False,
# #     fill_mode='nearest'
# )

In [3]:
# image_dir = "Datasets/BoundingBoxes/"

# for root, dirs, files in os.walk(image_dir):
#     for filename in files:
#         if not filename.startswith('.'):
#             if os.path.basename(root).startswith('subaru'):
#                 filepath = os.path.join(root, filename)
#                 # load the image and convert it to a numpy array
#                 img = load_img(filepath)
#                 img_array = img_to_array(img)
#                 # reshape the array to have an extra dimension
#                 img_array = img_array.reshape((1,) + img_array.shape)
#                 # generate X (in range) augmented images and save them
#                 for i in range(4):
#                     aug_img_array = datagen.flow(img_array, batch_size=1)[0]
#                     aug_img = aug_img_array.astype('uint8')
#                     new_filename = f"{filename.split('.')[0]}_aug{i}.jpg"
#                     new_filepath = os.path.join(root, new_filename)
#                     # save the augmented image
#                     Image.fromarray(aug_img.squeeze()).save(new_filepath)
#                     print(f"New file created: {new_filepath}")

### Generator (Data Generator)

In [4]:
image_dir = "Datasets/BoundingBoxes/"

In [5]:
def image_data_generator(image_dir, batch_size):
    images = []
    labels = []
    for brand_dir in os.listdir(image_dir):
        if not brand_dir.startswith('.'):
            for filename in os.listdir(os.path.join(image_dir, brand_dir)):
                if not filename.startswith('.'):
                    label = brand_dir
                    filepath = os.path.join(image_dir, brand_dir, filename)
                    image = cv2.imread(filepath)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    image = cv2.resize(image, (224, 224))
                    images.append(image)
                    labels.append(label)
                    if len(images) == batch_size:
                        labels = np.array(labels)
                        labels = np.where(np.array([label.split('_')[0] for label in labels]) == 'subaru', 1, 0)
                        yield np.array(images), np.array(labels)
                        images = []
                        labels = []
    if len(images) > 0:
        labels = np.array(labels)
        labels = np.where(np.array([label.split('_')[0] for label in labels]) == 'subaru', 1, 0)
        yield np.array(images), np.array(labels)

In [6]:
all_images = []
all_labels = []

for batch_images, batch_labels in image_data_generator(image_dir, batch_size=50000):
    all_images.append(batch_images)
    all_labels.append(batch_labels)

all_images = np.concatenate(all_images, axis=0)
all_labels = np.concatenate(all_labels, axis=0)

In [7]:
len(all_labels)

207368

In [8]:
1-(sum(all_labels)/len(all_labels)) ## baseline

0.6115601249951776

In [9]:
num_classes = len(np.unique(all_labels))
num_classes

2

### Train Test Split

To Train-Test split the data, we'll be seperating X on index to prevent all of the data from being loaded into the kernel concurrently.

In [10]:
idx = np.arange(len(all_images))
train_idx, test_idx = train_test_split(idx, test_size=0.2, stratify=all_labels)

In [11]:
X_train, y_train = all_images[train_idx], all_labels[train_idx]

In [12]:
X_test, y_test = all_images[test_idx], all_labels[test_idx]

In [13]:
# y_train = np.reshape(y_train, (-1, 1))
# y_test = np.reshape(y_test, (-1, 1))

In [14]:
len(y_test)

41474

In [15]:
print('Subaru count:', np.sum(y_train))
print('Other count:', np.sum(1 - y_train))

Subaru count: 64440
Other count: 101454


### Data Generators

We'll feed our data into the model in batches of 32 so that all images aren’t loaded into the kernel at the same time. Within this function, we'll divide X_low by 255, to normalize the data.

This operation is split into two seperate generators to prevent data leakage from the training set into the validation set. Since the function has been seperated, each function will only be called when needed.

In [16]:
def train_data_generator(X_train, y_train, batch_size):
    num_train_samples = len(X_train)
    train_indices = np.arange(num_train_samples)
    np.random.shuffle(train_indices)

    while True:
        for start_idx in range(0, num_train_samples, batch_size):
            end_idx = min(start_idx + batch_size, num_train_samples)
            batch_indices = train_indices[start_idx:end_idx]
            X_batch_train = torch.tensor(X_train[batch_indices], dtype=torch.float32)
            y_batch_train = torch.tensor(y_train[batch_indices], dtype=torch.long)

#             # Normalize the input data to [0, 1]
#             X_batch_train /= 255.0

            yield X_batch_train, y_batch_train

In [17]:
def test_data_generator(X_test, y_test, batch_size):
    num_test_samples = len(X_test)
    test_indices = torch.randperm(num_test_samples)
    while True:
        for start_idx_test in range(0, num_test_samples, batch_size):
            end_idx_test = min(start_idx_test + batch_size, num_test_samples)
            batch_indices_test = test_indices[start_idx_test:end_idx_test]
            batch_X_test = torch.tensor(X_test[batch_indices_test], dtype=torch.float32)
            batch_y_test = torch.tensor(y_test[batch_indices_test], dtype=torch.long)
            yield batch_X_test, batch_y_test

In [18]:
batch_size = 32
train_generator = train_data_generator(X_train, y_train, batch_size)
val_generator = test_data_generator(X_test, y_test, batch_size)

In [19]:
train_steps = len(X_train) // batch_size
val_steps = len(X_test) // batch_size

### Instantiating VGG16 Model (TensorFlow)

In [20]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
loss_fn = tf.keras.losses.BinaryCrossentropy()
metrics=['accuracy']
epochs = 50

In [21]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

Metal device set to: Apple M1 Max


2023-05-18 12:43:59.300818: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-18 12:43:59.301562: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [22]:
model = Sequential([
    base_model,
    Flatten(),
    Dense(1, activation='sigmoid')
])

In [23]:
model.compile(optimizer= optimizer,
              loss= loss_fn,
              metrics= metrics)

In [24]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10)


In [25]:
model.fit(train_generator,
          epochs=50,
          validation_data=val_generator,
          callbacks=[early_stopping],
          steps_per_epoch=train_steps, 
          validation_steps=val_steps)

Epoch 1/50


2023-05-18 12:44:04.388993: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-05-18 12:44:05.670503: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


5183/5184 [============================>.] - ETA: 0s - loss: 1.8217 - accuracy: 0.8267

2023-05-18 12:48:57.630811: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


5184/5184 [==============================] - 363s 69ms/step - loss: 1.8215 - accuracy: 0.8267 - val_loss: 2.3607 - val_accuracy: 0.7633
Epoch 2/50
5184/5184 [==============================] - 353s 68ms/step - loss: 1.4841 - accuracy: 0.8575 - val_loss: 1.1604 - val_accuracy: 0.8721
Epoch 3/50
5184/5184 [==============================] - 356s 69ms/step - loss: 1.3316 - accuracy: 0.8680 - val_loss: 2.8488 - val_accuracy: 0.8294
Epoch 4/50
5184/5184 [==============================] - 349s 67ms/step - loss: 1.3814 - accuracy: 0.8723 - val_loss: 1.2604 - val_accuracy: 0.8762
Epoch 5/50
5184/5184 [==============================] - 349s 67ms/step - loss: 1.3514 - accuracy: 0.8756 - val_loss: 1.2803 - val_accuracy: 0.8741
Epoch 6/50
5184/5184 [==============================] - 348s 67ms/step - loss: 1.3302 - accuracy: 0.8792 - val_loss: 1.4884 - val_accuracy: 0.8581
Epoch 7/50
5184/5184 [==============================] - 349s 67ms/step - loss: 1.3908 - accuracy: 0.8793 - val_loss: 1.2503 - val

In [27]:
def plot_cm(labels, predictions, p=0.5):
  cm = confusion_matrix(labels, predictions > p)
  plt.figure(figsize=(5,5))
  sns.heatmap(cm, annot=True, fmt="d")
  plt.title('Confusion matrix @{:.2f}'.format(p))
  plt.ylabel('Actual label')
  plt.xlabel('Predicted label')

  print('Legitimate Transactions Detected (True Negatives): ', cm[0][0])
  print('Legitimate Transactions Incorrectly Detected (False Positives): ', cm[0][1])
  print('Fraudulent Transactions Missed (False Negatives): ', cm[1][0])
  print('Fraudulent Transactions Detected (True Positives): ', cm[1][1])
  print('Total Fraudulent Transactions: ', np.sum(cm[1]))

In [28]:
cm = confusion_matrix(y_test, y_pred_labels)

# Create a heatmap using seaborn
plt.figure(figsize=(5, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)

NameError: name 'y_pred_labels' is not defined

In [38]:
cm

array([[ 7082, 18282],
       [  657, 15453]])

In [41]:
### Model results

1) ResNet150 Model
    - bounding boxes with multiple augmentations completed for subaru images
    - acc,prec,recall,f1_score (0.5754930800019289,0.47765162832387215,0.9923649906890131,0.6448971359419121)
    - baseline .625 (base_model = ResNet152(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    - converges to .999999 accuracy on model, .989 validation

In [29]:
model.save('models/Inception_v3_custom/')

INFO:tensorflow:Assets written to: models/Inception_v3_custom/assets


INFO:tensorflow:Assets written to: models/Inception_v3_custom/assets


In [30]:
save_path = "/Users/ryan/GA/Portfolio/Subaru-Seattle/models/savedModels/"
tf.saved_model.save(model, save_path)

INFO:tensorflow:Assets written to: /Users/ryan/GA/Portfolio/Subaru-Seattle/models/savedModels/assets


INFO:tensorflow:Assets written to: /Users/ryan/GA/Portfolio/Subaru-Seattle/models/savedModels/assets
